<a href="https://colab.research.google.com/github/AndersonxxAlves/Projeto-01/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get update && sudo apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh
import os
os.environ['OLLAMA_HOST'] = '0.0.0.0'
!fuser -k 11434/tcp || true # Mata qualquer processo na porta 11434, '|| true' para evitar erro se nenhum processo estiver na porta
!nohup ollama serve > ollama.log 2>&1 &
!sleep 5 # Give Ollama a few seconds to start
!ollama pull gpt-oss:120b-cloud

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [4,070 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,614 kB]
Fetched 5,812 kB in 4s (1,631 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (so

In [13]:
%%writefile app.py

import pandas as pd
import streamlit as st
import json
from pathlib import Path
import requests

MODELO = 'gpt-oss:120b-cloud'
OLLAMA_URL = 'http://localhost:11434/api/generate'

class BancoDados:
    def __init__(self, pasta_dados="."):
        self.pasta = Path(pasta_dados)
        self._carregar_dados()

    def _carregar_dados(self):
        self.saldos = pd.read_csv(self.pasta / "saldos_bancarios_ficticios.csv")
        self.extrato = pd.read_csv(self.pasta / "extrato_bancario_ficticio.csv")
        self.limites = pd.read_csv(self.pasta / "limites_cartoes_ficticios.csv")

        with open(self.pasta / "emprestimos_ficticios.json", encoding="utf-8") as f:
            self.emprestimos = pd.DataFrame(json.load(f))

        with open(self.pasta / "bloqueio_cartoes_ficticios.json", encoding="utf-8") as f:
            self.bloqueios = pd.DataFrame(json.load(f))

        with open(self.pasta / "segunda_via_boletos_ficticios.json", encoding="utf-8") as f:
            self.boletos = pd.DataFrame(json.load(f))

    def obter_saldo(self, cliente_id):
        return self.saldos[self.saldos["cliente_id"] == cliente_id]

    def obter_extrato(self, cliente_id):
        return self.extrato[self.extrato["cliente_id"] == cliente_id]

    def obter_limites_cartao(self, cliente_id):
        return self.limites[self.limites["cliente_id"] == cliente_id]

    def obter_emprestimos(self, cliente_id):
        return self.emprestimos[self.emprestimos["cliente_id"] == cliente_id]

    def obter_bloqueios_cartao(self, cliente_id):
        return self.bloqueios[self.bloqueios["cliente_id"] == cliente_id]

    def obter_boletos(self, cliente_id):
        return self.boletos[self.boletos["cliente_id"] == cliente_id]


class AssistenteBancario:
    def __init__(self, banco_dados):
        self.db = banco_dados

    def consultar_saldo(self, cliente_id):
        dados = self.db.obter_saldo(cliente_id)
        if dados.empty:
            return "Cliente não encontrado."
        saldo = dados.iloc[0]["saldo"]
        return f"Seu saldo atual é R$ {saldo:,.2f}"

    def consultar_limite_cartao(self, cliente_id):
        dados = self.db.obter_limites_cartao(cliente_id)
        if dados.empty:
            return "Limites não encontrados."
        disp = dados.iloc[0]["cartao_credito_disponivel"]
        return f"Seu limite disponível é R$ {disp:,.2f}"

    def consultar_status_bloqueio(self, cliente_id):
        dados = self.db.obter_bloqueios_cartao(cliente_id)
        if dados.empty:
            return "Nenhum bloqueio encontrado."
        status = dados.iloc[0]["status_bloqueio"]
        return f"Status do bloqueio do cartão: {status}"

    def gerar_contexto_cliente(self, cliente_id):
        saldo = self.db.obter_saldo(cliente_id)
        limite = self.db.obter_limites_cartao(cliente_id)
        bloqueio = self.db.obter_bloqueios_cartao(cliente_id)

        contexto = f"""
        Cliente ID: {cliente_id}

        SALDO:
        {saldo.to_dict(orient="records")}

        LIMITE CARTÃO:
        {limite.to_dict(orient="records")}

        BLOQUEIO:
        {bloqueio.to_dict(orient="records")}
        """
        return contexto

SYSTEM_PROMPT = """ Íris é uma agente virtual de assistência bancária criada para tornar a experiência financeira dos clientes mais simples, rápida e segura. Com uma comunicação clara, amigável e objetiva, ela entende as necessidades do usuário e oferece suporte imediato no dia a dia, ajudando a acompanhar informações importantes da conta e a resolver demandas comuns sem complicação.

Objetivo:
A Íris tem como objetivo principal oferecer ao cliente uma experiência bancária ágil, segura e autônoma para o acompanhamento e a gestão de suas principais demandas financeiras do dia a dia. Sua proposta é centralizar, em um único canal de atendimento, as interações mais recorrentes relacionadas à consulta de saldo, visualização de extrato, simulação de empréstimo, verificação de limite do cartão, emissão de segunda via de boletos e bloqueio de cartão.

REGRAS:
1 - Não realizar transações financeiras
Íris não deve efetuar transferências, pagamentos, investimentos ou qualquer movimentação de valores.

2 - Não alterar dados cadastrais do cliente
A agente não pode modificar informações pessoais, como endereço, telefone, e-mail ou dados bancários.

3 - Não fornecer aconselhamento financeiro personalizado
Íris não deve recomendar produtos financeiros, investimentos ou decisões de crédito com caráter consultivo.

4 - Não aprovar ou contratar produtos
A agente não tem permissão para efetivar contratação de empréstimos, cartões ou quaisquer serviços — apenas apresentar simulações informativas.

5 - Não compartilhar informações sem autenticação válida
Íris não deve exibir dados sensíveis caso a verificação de identidade do cliente não esteja concluída com sucesso.

6 - Não executar desbloqueio de cartão
A agente pode permitir o bloqueio por segurança, mas não deve realizar o desbloqueio do cartão.

7 - Não acessar ou expor dados de terceiros
Íris deve limitar as informações exclusivamente ao titular autenticado, sem qualquer exceção.

8 - Não sair do escopo funcional definido
A agente não deve responder ou executar solicitações fora de suas atribuições principais.

9 - Não armazenar informações sensíveis em conversas
Íris não deve solicitar nem reter senhas completas, códigos de segurança ou dados confidenciais desnecessários.

10 - Não fornecer garantias ou promessas
A agente não deve assegurar aprovação de crédito, prazos bancários ou resultados que dependam de análise sistêmica.
"""

def perguntar(msg, contexto_cliente):
    prompt = f"""
    {SYSTEM_PROMPT}

    CONTEXTO DO CLIENTE:
    {contexto_cliente}

    Perguntas: {msg}"""

    r = requests.post(OLLAMA_URL, json={"model": MODELO, "prompt": prompt, "stream": False})
    return r.json()['response']

st.title("Olá, Eu sou a Íris")

banco = BancoDados("/content/")
assistente = AssistenteBancario(banco)

if "cliente_id" not in st.session_state:
    st.session_state["cliente_id"] = 1001

cliente_id = st.session_state["cliente_id"]
contexto_cliente = assistente.gerar_contexto_cliente(cliente_id)

if pergunta := st.chat_input("Em que posso ajudar?"):
    st.chat_message("user").write(pergunta)
    with st.spinner("..."):
        st.chat_message("assistant").write(perguntar(pergunta, contexto_cliente))


Overwriting app.py


In [24]:
!fuser -k 8501/tcp || true # Mata qualquer processo na porta 8501
!killall ngrok || true # Mata qualquer processo ngrok existente para liberar o endpoint
!nohup streamlit run app.py --server.port 8501 --server.headless true > streamlit.log 2>&1 &
!sleep 10 # Dá um tempo maior para o Streamlit iniciar

# Instalação e configuração do Ngrok - Assumindo que o arquivo ngrok-v3-stable-linux-amd64.tgz foi manualmente carregado para /content/
!rm -f /usr/local/bin/ngrok || true # Remove qualquer versão antiga do ngrok
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz # Descompacta o arquivo tgz
!mv ngrok /usr/local/bin # Move o executável para o PATH
!ngrok authtoken 39sWSgqYFzn91GO7FrZtIhB3eA9_3M7tcwUghKxusNJEABiqN

# Inicia o ngrok em segundo plano e captura a URL
!nohup ngrok http 8501 > ngrok.log 2>&1 &
!sleep 15 # Dá um tempo MAIOR para o ngrok gerar o link e sua API iniciar
!cat ngrok.log # Check ngrok.log for issues/URL
!curl -s http://localhost:4040/api/tunnels # Get raw JSON output from ngrok API

8501/tcp:            79697
ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
{"tunnels":[{"name":"command_line","ID":"8269d6ac1cba790f9629d969c19288f6","uri":"/api/tunnels/command_line","public_url":"https://ungallantly-numinous-lyle.ngrok-free.dev","proto":"https","config":{"addr":"http://localhost:8501","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}
